In [54]:
from dbfread import DBF
import pandas as pd
import numpy as np

punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S')

punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME', 'TOTALTIME'])

in_punch_time = None
out_punch_time = None

for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = row['PDTIME']
    elif row['MODE'] == 1:
        out_punch_time = row['PDTIME']
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                pdate = [pd.to_datetime(in_punch_time, format='%d-%b-%y %H:%M:%S').strftime('%Y-%m-%d')]
                pdate_datetime = pd.to_datetime(pdate)
                if pdate_datetime.isin(result_df['PDATE']).any():
                    print("Date exists")
                else:

                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'PDATE': pdate,
                        'TOTALTIME': [f'{hours:02}:{minutes:02}:{seconds:02}']
                    })], ignore_index=True)

result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

result_df.to_csv('./5thJan_punches.csv', index=False)

['2023-09-01']
['2023-09-02']
['2023-09-04']
['2023-09-05']
['2023-09-06']
['2023-09-07']
['2023-09-08']
['2023-09-09']
['2023-09-11']
['2023-09-12']
['2023-09-13']
['2023-09-14']
['2023-09-15']
['2023-09-16']
['2023-09-18']
['2023-09-19']
['2023-09-20']
['2023-09-21']
['2023-09-22']
['2023-09-23']
['2023-09-25']
['2023-09-26']
['2023-09-27']
['2023-09-28']
['2023-09-29']
['2023-09-30']
['2023-09-03']


In [59]:
from dbfread import DBF
import pandas as pd
import numpy as np

punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S')

punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME'])

in_punch_time = None
out_punch_time = None

for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = row['PDTIME']
    elif row['MODE'] == 1:
        out_punch_time = row['PDTIME']
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                result_df = pd.concat([result_df, pd.DataFrame({
                    'TOKEN': [row['TOKEN']],
                    'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                    'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                    'PDATE': [pd.to_datetime(in_punch_time, format='%d-%b-%y %H:%M:%S').strftime('%Y-%m-%d')]
                })], ignore_index=True)

result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

import pandas as pd

# Assuming result_df['PDATE'] and result_df['TOKEN'] are columns of interest
result_df['PDATE'] = pd.to_datetime(result_df['PDATE'])

# List of columns to check for duplicates
columns_to_check = ['PDATE', 'TOKEN']

# Check for duplicate rows based on 'PDATE' and 'TOKEN'
duplicate_rows = result_df[result_df.duplicated(columns_to_check)]

if not duplicate_rows.empty:
    print("Duplicate rows found based on 'PDATE' and 'TOKEN':")
    print(duplicate_rows)
else:
    print("No duplicate rows found based on 'PDATE' and 'TOKEN'")


result_df.to_csv('./5thJan_punches.csv', index=False)

Duplicate rows found based on 'PDATE' and 'TOKEN':
   TOKEN      PDATE               INTIME              OUTTIME TOTALTIME
29     2 2023-09-03  2023-09-03 16:00:59  2023-09-04 02:01:00  10:00:00
31     2 2023-09-04  2023-09-04 18:00:00  2023-09-05 05:00:00  11:00:00
33     2 2023-09-05  2023-09-05 23:01:00  2023-09-06 04:00:59  04:59:59


In [76]:
import pandas as pd
from dbfread import DBF
import numpy as np

muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))

muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 
                         'EMP_DEPT','DEPT_NAME', 'EMP_DESI',
                         'DESI_NAME']]
muster_df = muster_df.sort_values(by=['EMPCODE'])

punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S')

punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME', 'INTIME1', 'OUTTIME1', 'TOTALTIME'])

in_punch_time = None
out_punch_time = None

for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = row['PDTIME']
    elif row['MODE'] == 1:
        out_punch_time = row['PDTIME']
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME1': [np.nan],
                        'OUTTIME1': [np.nan],
                        'TOTALTIME': [f'{hours:02}:{minutes:02}:{seconds:02}']
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    result_df.loc[duplicates.index[-1], 'INTIME1'] = in_punch_time.strftime('%Y-%m-%d %H:%M:%S')
                    result_df.loc[duplicates.index[-1], 'OUTTIME1'] = out_punch_time.strftime('%Y-%m-%d %H:%M:%S')
                    
                    # Calculate TOTALTIME based on the sum of time differences for both pairs
                    total_time_difference_1 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME'])
                    total_time_difference_2 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME1']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME1'])
                    
                    total_hours_1, total_remainder_1 = divmod(total_time_difference_1.seconds, 3600)
                    total_minutes_1, total_seconds_1 = divmod(total_remainder_1, 60)
                    
                    total_hours_2, total_remainder_2 = divmod(total_time_difference_2.seconds, 3600)
                    total_minutes_2, total_seconds_2 = divmod(total_remainder_2, 60)
                    
                    total_hours = total_hours_1 + total_hours_2
                    total_minutes = total_minutes_1 + total_minutes_2
                    total_seconds = total_seconds_1 + total_seconds_2
                    
                    result_df.loc[duplicates.index[-1], 'TOTALTIME'] = f'{total_hours:02}:{total_minutes:02}:{total_seconds:02}'

result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])


final_merged_df = pd.merge(muster_df, result_df, on='TOKEN', how='left')
final_merged_df = final_merged_df.sort_values(by=['EMPCODE', 'TOKEN', 'PDATE'])
final_merged_df.to_csv('./final.csv', index=False)


# result_df.to_csv('./6thJan_punches_modified.csv', index=False)


In [146]:
import pandas as pd
from dbfread import DBF
import numpy as np

muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))

muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 
                         'EMP_DEPT','DEPT_NAME', 'EMP_DESI',
                         'DESI_NAME']]
muster_df = muster_df.sort_values(by=['EMPCODE'])

punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S')

punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME', 'TOTALTIME', 'INTIME1'])

in_punch_time = None
out_punch_time = None

for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = row['PDTIME']
    elif row['MODE'] == 1:
        out_punch_time = row['PDTIME']
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'TOTALTIME': [f'{hours:02}:{minutes:02}:{seconds:02}'],
                        'INTIME1': [np.nan]
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    result_df.loc[duplicates.index[-1], 'OUTTIME'] = out_punch_time.strftime('%Y-%m-%d %H:%M:%S')
                    
                    # Calculate TOTALTIME based on the sum of time differences for both pairs
                    total_time_difference_1 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME'])
                    
                    total_hours_1, total_remainder_1 = divmod(total_time_difference_1.seconds, 3600)
                    total_minutes_1, total_seconds_1 = divmod(total_remainder_1, 60)
                    
                    total_hours = total_hours_1
                    total_minutes = total_minutes_1
                    total_seconds = total_seconds_1
                    
                    result_df.loc[duplicates.index[-1], 'TOTALTIME'] = f'{total_hours:02}:{total_minutes:02}:{total_seconds:02}'

# Drop 'INTIME1' if it was added
if 'INTIME1' in result_df.columns:
    result_df = result_df.drop(columns=['INTIME1'])

result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])
result_df.to_csv('./6thJan_punches_modified.csv', index=False)


In [150]:
import pandas as pd
from dbfread import DBF
import numpy as np

# Load and process muster data
muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))
muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 'EMP_DEPT', 'DEPT_NAME', 'EMP_DESI', 'DESI_NAME']]
muster_df = muster_df.sort_values(by=['EMPCODE'])

# Load and process punches data
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

# Make seconds part '00' in PDTIME
punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort punches_df based on TOKEN, PDTIME, MODE
punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

# Initialize result DataFrame
result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME', 'INTIME1', 'OUTTIME1'])

in_punch_time = None
out_punch_time = None

# Iterate through punches_df to process punch data
for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
    elif row['MODE'] == 1:
        out_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME1': [np.nan],
                        'OUTTIME1': [np.nan]
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    result_df.loc[duplicates.index[-1], 'INTIME1'] = in_punch_time.strftime('%Y-%m-%d %H:%M:%S')
                    result_df.loc[duplicates.index[-1], 'OUTTIME1'] = out_punch_time.strftime('%Y-%m-%d %H:%M:%S')
                    
                    # Calculate TOTALTIME based on the sum of time differences for both pairs
                    total_time_difference_1 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME'])
                    total_time_difference_2 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME1']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME1'])
                    

# Sort result_df based on TOKEN, PDATE
result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

# Save the result to a CSV file
result_df.to_csv('./6thJan_punches_modified.csv', index=False)

In [165]:
import pandas as pd
from dbfread import DBF
import numpy as np

# Load and process muster data
muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))
muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 'EMP_DEPT', 'DEPT_NAME', 'EMP_DESI', 'DESI_NAME']]
muster_df = muster_df.sort_values(by=['EMPCODE'])

# Load and process punches data
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

# Make seconds part '00' in PDTIME
punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort punches_df based on TOKEN, PDTIME, MODE
punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

# Initialize result DataFrame with dynamic column names
result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 
                                  'OUTTIME'])

in_punch_time = None
out_punch_time = None

# Iterate through punches_df to process punch data
for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
    elif row['MODE'] == 1:
        out_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')]
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    num_pairs = sum([col.startswith('INTIME') for col in duplicates.columns]) + 1
                    
                    # Add additional columns for each in and out punch pair dynamically
                    result_df = pd.concat([result_df, pd.DataFrame({
                        f'INTIME{num_pairs}': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        f'OUTTIME{num_pairs}': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                    })], axis=1)
                    
                    

# Sort result_df based on TOKEN, PDATE
result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

# Save the result to a CSV file
result_df.to_csv('./6thJan_punches_modified.csv', index=False)


In [179]:
import pandas as pd
from dbfread import DBF
import numpy as np

# Load and process punches data
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

# Make seconds part '00' in PDTIME
punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort punches_df based on TOKEN, PDTIME, MODE
punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

# Initialize result DataFrame with dynamic column names
result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 
                                  'OUTTIME'])

in_punch_time = None
out_punch_time = None

# Iterate through punches_df to process punch data
for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
    elif row['MODE'] == 1:
        out_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')]
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    num_pairs = sum([col.startswith('INTIME') for col in duplicates.columns]) + 1
                    
                    # Add additional columns for each in and out punch pair dynamically
                    result_df = pd.concat([result_df, pd.DataFrame({
                        f'INTIME{num_pairs}': [in_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                        f'OUTTIME{num_pairs}': [out_punch_time.strftime('%Y-%m-%d %H:%M:%S')],
                    })], axis=1)
                    
                    

# Sort result_df based on TOKEN, PDATE
result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

# Save the result to a CSV file
result_df.to_csv('./6thJan_punches_modified.csv', index=False)


In [192]:
import pandas as pd
from dbfread import DBF
import numpy as np

# Load and process muster data
muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))
muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 'EMP_DEPT', 'DEPT_NAME', 'EMP_DESI', 'DESI_NAME']]
muster_df = muster_df.sort_values(by=['EMPCODE'])

# Load and process punches data
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

# Make seconds part '00' in PDTIME
punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort punches_df based on TOKEN, PDTIME, MODE
punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

# Initialize result DataFrame
result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME', 'INTIME1', 'OUTTIME1', 'TOTALTIME'])

in_punch_time = None
out_punch_time = None

# Iterate through punches_df to process punch data
for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
    elif row['MODE'] == 1:
        out_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME1': [np.nan],
                        'OUTTIME1': [np.nan],
                        'TOTALTIME': [f'{hours:02}:{minutes:02}']
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    result_df.loc[duplicates.index[-1], 'INTIME1'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                    result_df.loc[duplicates.index[-1], 'OUTTIME1'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    
                    # Calculate TOTALTIME based on the sum of time differences for both pairs
                    total_time_difference_1 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME'])
                    total_time_difference_2 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME1']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME1'])
                    
                    total_time_difference = total_time_difference_1 + total_time_difference_2
                    
                    total_hours, total_remainder = divmod(total_time_difference.seconds, 3600)
                    total_minutes, _ = divmod(total_remainder, 60)
                    
                    result_df.loc[duplicates.index[-1], 'TOTALTIME'] = f'{total_hours:02}:{total_minutes:02}'

# Sort result_df based on TOKEN, PDATE
result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

# Save the result to a CSV file
result_df.to_csv('./6thJan_punches_modified.csv', index=False)

In [1]:
import pandas as pd
from dbfread import DBF
import numpy as np

# Load and process muster data
muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))
muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 'EMP_DEPT', 'DEPT_NAME', 'EMP_DESI', 'DESI_NAME']]
muster_df = muster_df.sort_values(by=['EMPCODE'])

# Load and process punches data
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))
punches_df = punches_df.sort_values(by=['TOKEN', 'PDATE', 'MODE'])

# Make seconds part '00' in PDTIME
punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort punches_df based on TOKEN, PDTIME, MODE
punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

# Initialize result DataFrame
result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME', 'OUTTIME', 'INTIME1', 'OUTTIME1', 'INTIME2', 'OUTTIME2', 'INTIME3', 'OUTTIME3','TOTALTIME'])

in_punch_time = None
out_punch_time = None

# Iterate through punches_df to process punch data
for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
    elif row['MODE'] == 1:
        out_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME1': [np.nan],
                        'OUTTIME1': [np.nan],
                        'INTIME2': [np.nan],
                        'OUTTIME2': [np.nan],
                        'INTIME3': [np.nan],
                        'OUTTIME3': [np.nan],
                        'TOTALTIME': [f'{hours:02}:{minutes:02}']
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    if pd.isna(result_df.loc[duplicates.index[-1], 'INTIME1']):
                        result_df.loc[duplicates.index[-1], 'INTIME1'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME1'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    elif pd.isna(result_df.loc[duplicates.index[-1], 'INTIME2']):
                        result_df.loc[duplicates.index[-1], 'INTIME2'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME2'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    elif pd.isna(result_df.loc[duplicates.index[-1], 'INTIME3']):
                        result_df.loc[duplicates.index[-1], 'INTIME3'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME3'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    else:
                        # If INTIME1, OUTTIME1, INTIME2, OUTTIME2, INTIME3, and OUTTIME3 are not NaN, update them
                        result_df.loc[duplicates.index[-1], 'INTIME1'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME1'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                        
                        result_df.loc[duplicates.index[-1], 'INTIME2'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME2'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                        
                        result_df.loc[duplicates.index[-1], 'INTIME3'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME3'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    
                    # Calculate TOTALTIME based on the sum of time differences for all pairs

                    total_time_difference_1 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME'])
                    total_time_difference_2 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME1']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME1'])
                    total_time_difference_3 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME2']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME2'])
                    total_time_difference_4 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME3']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME3'])
                    
                    # Sum only non-NaN values of type pd.Timedelta
                    total_time_difference = pd.to_timedelta(0)

                    if isinstance(total_time_difference_1, pd.Timedelta):
                        total_time_difference += total_time_difference_1

                    if isinstance(total_time_difference_2, pd.Timedelta):
                        total_time_difference += total_time_difference_2

                    if isinstance(total_time_difference_3, pd.Timedelta):
                        total_time_difference += total_time_difference_3

                    if isinstance(total_time_difference_4, pd.Timedelta):
                        total_time_difference += total_time_difference_4

                    # Calculate hours and minutes
                    total_hours, total_remainder = divmod(total_time_difference.seconds, 3600)
                    total_minutes, _ = divmod(total_remainder, 60)

                    result_df.loc[duplicates.index[-1], 'TOTALTIME'] = f'{total_hours:02}:{total_minutes:02}'

# Sort result_df based on TOKEN, PDATE
result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

# Save the result to a CSV file
result_df.to_csv('./6thJan_punches_modified.csv', index=False)

In [11]:
import pandas as pd
from dbfread import DBF
import numpy as np

# Load and process muster data
muster_table = DBF('D:/ZIONtest/muster.dbf', load=True)
muster_df = pd.DataFrame(iter(muster_table))
muster_df = muster_df[['TOKEN', 'COMCODE', 'NAME', 'EMPCODE', 'EMP_DEPT', 'DEPT_NAME', 'EMP_DESI', 'DESI_NAME']]
muster_df = muster_df.sort_values(by=['TOKEN'])

# Load and process punches data
punches_table = DBF('D:/ZIONtest/punches.dbf', load=True)
punches_df = pd.DataFrame(iter(punches_table))

# Make seconds part '00' in PDTIME
punches_df['PDTIME'] = pd.to_datetime(punches_df['PDTIME'], format='%d-%b-%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort punches_df based on TOKEN, PDTIME, MODE
punches_df.sort_values(by=['TOKEN', 'PDTIME', 'MODE'], inplace=True)

# Initialize result DataFrame
result_df = pd.DataFrame(columns=['TOKEN', 'PDATE', 'INTIME1', 'OUTTIME1', 'INTIME2', 'OUTTIME2', 'INTIME3', 'OUTTIME3', 'INTIME4', 'OUTTIME4','INTIME', 'OUTTIME','TOTALTIME', 'REMARKS'])

in_punch_time = None
out_punch_time = None

# Iterate through punches_df to process punch data
for index, row in punches_df.iterrows():
    if row['MODE'] == 0:
        in_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
    elif row['MODE'] == 1:
        out_punch_time = pd.to_datetime(row['PDTIME']).replace(second=0)
        if in_punch_time is not None:
            time_difference = out_punch_time - in_punch_time
            if time_difference.total_seconds() > 0:
                hours, remainder = divmod(time_difference.seconds, 3600)
                minutes, seconds = divmod(remainder, 60)
                
                # Check for duplicates based on 'PDATE' and 'TOKEN'
                duplicates = result_df[(result_df['PDATE'] == in_punch_time.strftime('%Y-%m-%d')) & (result_df['TOKEN'] == row['TOKEN'])]
                
                if duplicates.empty:
                    # No duplicates, add a new row
                    result_df = pd.concat([result_df, pd.DataFrame({
                        'TOKEN': [row['TOKEN']],
                        'PDATE': [in_punch_time.strftime('%Y-%m-%d')],
                        'INTIME1': [in_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'OUTTIME1': [out_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'INTIME2': [np.nan],
                        'OUTTIME2': [np.nan],
                        'INTIME3': [np.nan],
                        'OUTTIME3': [np.nan],
                        'INTIME4': [np.nan],
                        'OUTTIME4': [np.nan],
                        'INTIME': [in_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'OUTTIME': [out_punch_time.strftime('%Y-%m-%d %H:%M')],
                        'TOTALTIME': [f'{hours:02}:{minutes:02}'],
                        'REMARKS': ""
                    })], ignore_index=True)
                else:
                    # Duplicates found, update the existing row
                    if pd.isna(result_df.loc[duplicates.index[-1], 'INTIME2']):
                        result_df.loc[duplicates.index[-1], 'INTIME2'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME2'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    elif pd.isna(result_df.loc[duplicates.index[-1], 'INTIME3']):
                        result_df.loc[duplicates.index[-1], 'INTIME3'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME3'] = out_punch_time.strftime('%Y-%m-%d %H:%M')
                    elif pd.isna(result_df.loc[duplicates.index[-1], 'INTIME4']):
                        result_df.loc[duplicates.index[-1], 'INTIME4'] = in_punch_time.strftime('%Y-%m-%d %H:%M')
                        result_df.loc[duplicates.index[-1], 'OUTTIME4'] = out_punch_time.strftime('%Y-%m-%d %H:%M')

                    # update REMARKS for multiple punches
                    result_df.loc[duplicates.index[-1], 'REMARKS'] = "*"

                    
                    # Update OUTTIME4
                    result_df.loc[duplicates.index[-1], 'OUTTIME'] = out_punch_time.strftime('%Y-%m-%d %H:%M') if not pd.isna(out_punch_time) else np.nan


                    # Calculate TOTALTIME based on the sum of time differences for all pairs
                    total_time_difference_1 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME1']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME1'])
                    total_time_difference_2 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME2']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME2'])
                    total_time_difference_3 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME3']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME3'])
                    total_time_difference_4 = pd.to_datetime(result_df.loc[duplicates.index[-1], 'OUTTIME4']) - pd.to_datetime(result_df.loc[duplicates.index[-1], 'INTIME4'])
                    
                    # Sum only non-NaN values of type pd.Timedelta
                    total_time_difference = pd.to_timedelta(0)

                    if isinstance(total_time_difference_1, pd.Timedelta):
                        total_time_difference += total_time_difference_1

                    if isinstance(total_time_difference_2, pd.Timedelta):
                        total_time_difference += total_time_difference_2

                    if isinstance(total_time_difference_3, pd.Timedelta):
                        total_time_difference += total_time_difference_3

                    if isinstance(total_time_difference_4, pd.Timedelta):
                        total_time_difference += total_time_difference_4

                    # Calculate hours and minutes
                    total_hours, total_remainder = divmod(total_time_difference.seconds, 3600)
                    total_minutes, _ = divmod(total_remainder, 60)

                    result_df.loc[duplicates.index[-1], 'TOTALTIME'] = f'{total_hours:02}:{total_minutes:02}'

# Sort result_df based on TOKEN, PDATE
result_df = result_df.sort_values(by=['TOKEN', 'PDATE'])

merged_df = pd.merge(muster_df, result_df, on='TOKEN', how='left')

# Save the result to a CSV file
merged_df.to_csv('./6thJan_punches_modified.csv', index=False)